In [ ]:
import calendar
import json
import requests
import csv

filename = "output.csv"
symbols = [ "GME", "AMC", "KOSS", "NAKD", "BBBY", "NOK", "GM", "AAPL", "TSLA", "MSFT", "SPY", "SPXS" ]


finraUrl = "https://api.finra.org/data/group/OTCMarket/name/weeklySummary"

header = 'issueSymbolIdentifier,issueName,totalWeeklyShareQuantity,totalWeeklyTradeCount,marketParticipantName,tierIdentifier,weekStartDate\n'

with open(filename, "w") as f:
    f.seek(0)
    f.write(header)
    f.truncate()

for year in [2019,2020,2021]:
    for month in range(1, 12):
        startDate = str(year) + "-" + str(month).zfill(2) + "-" + "01"
        endDate = str(year) + "-" + str(month).zfill(2) + "-" + str(calendar._monthlen(year, month))
        print(startDate)
        print(endDate)

        tempQuery = {
        "fields":["issueSymbolIdentifier", "issueName", "totalWeeklyShareQuantity","totalWeeklyTradeCount","marketParticipantName","tierIdentifier","weekStartDate"],  
        "limit" : 50000,
        "domainFilters": [{
        "fieldName" : "issueSymbolIdentifier",
        "values" : symbols
        }],
        "dateRangeFilters": [ {

            "startDate" : startDate,

            "endDate" : endDate,

            "fieldName" : "weekStartDate"

        } ]
        }

        query = json.dumps(tempQuery)

        r = requests.post(finraUrl, query)
        with open(filename, 'a') as f:
            reader = csv.DictReader(r.text.splitlines(), delimiter=',')
            writer = csv.DictWriter(f, reader.fieldnames)
            writer.writerows(reader)



In [ ]:
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
import datetime

yf.pdr_override()
df = pd.read_csv (r'output.csv')

df = df.drop(columns=['issueName', 'totalWeeklyShareQuantity','totalWeeklyTradeCount','marketParticipantName','tierIdentifier'])
df = df.sort_values(by=['issueSymbolIdentifier','weekStartDate'])
df = df.drop_duplicates()
symbols = df["issueSymbolIdentifier"].drop_duplicates()

priceColumns = ['issueSymbolIdentifier','weekStartDate','High', 'Low', 'Open', 'Close', 'Volume']
priceData = pd.DataFrame(columns = priceColumns)
priceData.to_csv('price.csv',index=False)
for symbol in symbols:
        curr = df[df["issueSymbolIdentifier"].isin([symbol])]
        for x in curr['weekStartDate']:
                startDate = datetime.date.fromisoformat(x)
                endDate = startDate + datetime.timedelta( (5-startDate.weekday()) % 7 ) #yahoo finance ignores the endDate in the result set.
                output = pdr.get_data_yahoo(symbol,startDate,endDate)
                highPrice = output['High'].max()
                lowPrice = output['Low'].min()
                openPrice = output['Open'].mean()
                closePrice = output['Close'].mean()
                volume = output['Volume'].sum()

                newRow = {
                        'issueSymbolIdentifier':symbol,
                        'weekStartDate': startDate.isoformat(),
                        'High': highPrice,
                        'Low': lowPrice,
                        'Open': openPrice,
                        'Close': closePrice,
                        'Volume': volume
                        }
                priceData = pd.DataFrame(columns = priceColumns)
                priceData = priceData.append(newRow, ignore_index="True")

                del startDate
                del endDate
                del output
                del highPrice
                del lowPrice
                del openPrice
                del closePrice
                del volume
                del newRow
                priceData.to_csv('price.csv',index=False,header=False,mode='a')
                del priceData
        del curr